# ***Scandinavian News Articles***

This notebook is a part of the final project in *Computational Tools for data science 02807*

### **Imports**

In [ ]:
import pandas as pd 

### **Import Datasets**

The datasets have been scraped using ..

In [6]:
norwegian_articles = pd.read_csv('./data/norway_all_articles_classified.csv')
danish_articles = pd.read_csv('./data/denmark_all_articles_classified.csv')
swedish_articles = pd.read_csv('./data/sweden_all_articles_classified.csv')

norwegian_articles.head(3)

,date,headline,content,url,source,country,primary_agenda,primary_category,political_categories,cultural_categories,political_score,cultural_score
0,2025-11-05,Sykefraværet stupte: Her jobber de fire dagers...,Velkommen til drømme-barnehagen! Plenty ansatt...,https://www.vg.no/nyheter/i/MnMr5K/80-100-ordn...,vg,norway,cultural,cultural_identity,elections,"cultural_identity,arts",2,2
1,2025-10-12,Østers-misjonæren som må kaste alt,Et kloakkutslipp i Glomma har stanset østershø...,https://www.vg.no/nyheter/i/RzPMed/oestersprod...,vg,norway,political,elections,"elections,defense",NaN,3,0
2,2025-10-04,Forsvarer landslagsprofil etter rasismeanklage...,Aron Dønnum er blant spillerne som er tatt med...,https://www.nrk.no/sport/martin-odegaard-er-ik...,nrk,norway,cultural,arts,defense,arts,1,1
